In [196]:
import requests
import json
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import datetime

In [2]:
config_file = "config.json"
with open(config_file, "r") as input_file:
    config = json.load(input_file)

In [11]:
# query 24 hours
start_timestamp = 1692619200 # unix epoch in utc
interval = 900 # timestep in seconds (= 15 minutes)
number_intervals = 96
query_times = [start_timestamp + i*interval for i in range(0, 96)]

In [13]:
# read consumer data
def query_consumers():
    request_consumers = f"{config['consumer_api']}/consumer/all"
    result = requests.get(request_consumers)
    consumers = json.loads(result.text)
    return consumers

# read network data
def query_network():
    initialize_network = f"{config['network_api']}/initialize"
    request_initialize = requests.get(initialize_network)

    request_network = f"{config['network_api']}/network"
    result = requests.get(request_network)
    network = json.loads(result.text)
    return network

def query_range(queries):
    opfs = {}
    for ts in queries:
        request_opf = f"{config['network_api']}/opf/{ts}"
        result = requests.get(request_opf)
        opfs[ts] = json.loads(result.text)
    return opfs

In [178]:
def get_consumer_lat_lon(identifier, consumers):
    return [{ 'latitude': x['latitude'], 'longitude': x['longitude']}  for x in consumers if x['identifier'] == identifier][0]

def get_network_entity_lat_lon(identifier, entities):
    return [{ 'latitude': x['latitude'], 'longitude': x['longitude']} for x in entities if x['identifier'] == identifier][0]

def get_nodes(consumers, network) -> pd.DataFrame:
    data = {}
    data['name'] = [x['name'] for x in consumers]
    data['identifier'] = [x['identifier'] for x in consumers]
    data['latitude'] = [x['latitude'] for x in consumers]
    data['longitude'] = [x['longitude'] for x in consumers]
    data['category'] = [x['category'] for x in consumers]


    data['name'] += ([x['name'] for x in network['entities']])
    data['identifier'] += ([x['identifier'] for x in network['entities']])
    data['latitude'] += ([x['latitude'] for x in network['entities']])
    data['longitude'] += ([x['longitude'] for x in network['entities']])
    data['category'] += [x['category'] for x in network['entities']]

    return pd.DataFrame(data=data)

def get_line_connections(consumers, network) -> pd.DataFrame:
    latitudes = []
    longitudes = []
    identifiers = []

    consumer_ids = [c['identifier'] for c in consumers]
    network_entity_ids = [n['identifier'] for n in network['entities']]
    print(consumer_ids)
    print(network_entity_ids)
    for line in network['lines']:
        fb = line['from_bus']
        tb = line['to_bus']        
        if fb in consumer_ids:
            lt = get_consumer_lat_lon(fb, consumers)
            longitudes.append(lt['longitude'])
            latitudes.append(lt['latitude'])
            identifiers.append(fb)
        elif fb in network_entity_ids:
            lt = get_network_entity_lat_lon(fb, network['entities'])
            longitudes.append(lt['longitude'])
            latitudes.append(lt['latitude'])
            identifiers.append(fb)
        else:
            print(f"identifier {fb} not found")
        if tb in consumer_ids:
            lt = get_consumer_lat_lon(tb, consumers)
            longitudes.append(lt['longitude'])
            latitudes.append(lt['latitude'])
            identifiers.append(tb)            
        elif tb in network_entity_ids:
            lt = get_network_entity_lat_lon(tb, network['entities'])
            longitudes.append(lt['longitude'])
            latitudes.append(lt['latitude'])
            identifiers.append(tb)            
        else:
            print(f"identifier {tb} not found")
        print(line)
    data = { 'identifier': identifiers, 'longitude': longitudes, 'latitude': latitudes }
    return pd.DataFrame(data=data)

def create_node_identifier_mapping(nodes_dataframe):
    t = nodes_dataframe.reset_index()
    return dict(zip(t.index, t.identifier))
    
# # create load dataset
# def create_load_dataframe(consumers, network, opf):
#     load = opf['load']
#     data = {}
#     identifiers = [v for k, v in load['name'].items()]
#     #print(identifiers)
#     reverse_map ={ v: k for k, v in load['name'].items() }
#     loads = [load['p_mw'][reverse_map[k]] for k in identifiers]
#     #print(loads)
#     data['identifier'] = identifiers
#     data['load'] = loads
#     return pd.DataFrame(data=data)


In [14]:
consumers = query_consumers()
network = query_network()
opfs = query_range(query_times)

In [19]:
nodes_df = get_nodes(consumers, network)
lines_df = get_line_connections(consumers, network)


['e6c8200f-d84b-499b-ad13-4313eec39ca2', '60d36bfa-92a2-4521-95fa-ed0e95f2413f', 'ec28306c-772c-4d65-9916-55c03ce61566']
['d3ae8923-d4cd-46fb-ae6c-e4a20c7e5dab']
{'from_bus': 'ec28306c-772c-4d65-9916-55c03ce61566', 'to_bus': '60d36bfa-92a2-4521-95fa-ed0e95f2413f', 'std_type': 'NAYY 4x50 SE', 'length_km': 0.1}
{'from_bus': '60d36bfa-92a2-4521-95fa-ed0e95f2413f', 'to_bus': 'e6c8200f-d84b-499b-ad13-4313eec39ca2', 'std_type': 'NAYY 4x50 SE', 'length_km': 0.1}
{'from_bus': 'e6c8200f-d84b-499b-ad13-4313eec39ca2', 'to_bus': 'd3ae8923-d4cd-46fb-ae6c-e4a20c7e5dab', 'std_type': 'NAYY 4x50 SE', 'length_km': 0.1}


In [20]:
nodes_df

,name,identifier,latitude,longitude,category
0,location 1,e6c8200f-d84b-499b-ad13-4313eec39ca2,48.3744,14.5123,household
1,location 2,60d36bfa-92a2-4521-95fa-ed0e95f2413f,48.3744,14.5115,household
2,location 3,ec28306c-772c-4d65-9916-55c03ce61566,48.3740,14.5111,household
3,grid connection,d3ae8923-d4cd-46fb-ae6c-e4a20c7e5dab,48.3744,14.5131,network


In [22]:
lines_df

,identifier,longitude,latitude
0,ec28306c-772c-4d65-9916-55c03ce61566,14.5111,48.3740
1,60d36bfa-92a2-4521-95fa-ed0e95f2413f,14.5115,48.3744
2,60d36bfa-92a2-4521-95fa-ed0e95f2413f,14.5115,48.3744
3,e6c8200f-d84b-499b-ad13-4313eec39ca2,14.5123,48.3744
4,e6c8200f-d84b-499b-ad13-4313eec39ca2,14.5123,48.3744
5,d3ae8923-d4cd-46fb-ae6c-e4a20c7e5dab,14.5131,48.3744


In [203]:
# read consumer data
request_consumers = f"{config['consumer_api']}/consumer/all"
result = requests.get(request_consumers)
consumers = json.loads(result.text)
# read network data
initialize_network = f"{config['network_api']}/initialize"
request_initialize = requests.get(initialize_network)

request_network = f"{config['network_api']}/network"
result = requests.get(request_network)
network = json.loads(result.text)

unix_time_seconds = 1692619200
request_opf = f"{config['network_api']}/opf/{unix_time_seconds}"
result = requests.get(request_opf)
opf = json.loads(result.text)

# create pandas data frame
data = {}
data['name'] = [x['name'] for x in consumers]
data['identifier'] = [x['identifier'] for x in consumers]
data['latitude'] = [x['latitude'] for x in consumers]
data['longitude'] = [x['longitude'] for x in consumers]
data['category'] = [x['category'] for x in consumers]


data['name'] += ([x['name'] for x in network['entities']])
data['identifier'] += ([x['identifier'] for x in network['entities']])
data['latitude'] += ([x['latitude'] for x in network['entities']])
data['longitude'] += ([x['longitude'] for x in network['entities']])
data['category'] += [x['category'] for x in network['entities']]

node_df = pd.DataFrame(data=data)
bus_df = pd.DataFrame(data=opf['bus'])
load_df = pd.DataFrame(data=opf['load'])
line_df = pd.DataFrame(data=opf['line'])
res_bus = pd.DataFrame(data=opf['res_bus'])
res_line = pd.DataFrame(data=opf['res_line'])
res_load = pd.DataFrame(data=opf['res_load'])
res_ext_grid = pd.DataFrame(data=opf['res_ext_grid'])

In [169]:
# create load dataset
def create_load_dataframe(consumers, network, opf):
    load = opf['load']
    data = {}
    identifiers = [v for k, v in load['name'].items()]
    #print(identifiers)
    reverse_map ={ v: k for k, v in load['name'].items() }
    loads = [load['p_mw'][reverse_map[k]] for k in identifiers]
    #print(loads)
    data['identifier'] = identifiers
    data['load'] = loads
    return pd.DataFrame(data=data)

# create consumer load timeseries
def load_ts(nodes_df, opfs):
    node_identifier_map = create_node_identifier_mapping(nodes_df)
    opf_keys = [k for k in sorted(opfs.keys())]
    dataframes = []
    for ok in opf_keys:
        o = opfs[ok]
        df1 = pd.DataFrame(o['res_load'])
        df1 = df1.reset_index()
        df1['node_id'] = df1['index'].astype('int64')
        #df2 = nodes_df.reset_index()
        #df2['idx'] =df2['index'].astype('int64')
        #dataframe = df1.join(df2, on='idx')
        dataframe = df1
        dataframe['time'] = datetime.datetime.fromtimestamp(ok)
        dataframes.append(dataframe)
    res_load_ts = pd.concat(dataframes).reset_index()[['node_id', 'p_mw', 'q_mvar', 'time']]
    res_load_ts['identifier'] = res_load_ts['node_id'].map(node_identifier_map)
    res_load_ts = res_load_ts[['p_mw', 'q_mvar', 'time', 'identifier']]
    return res_load_ts
   
def bus_ts(nodes_df, opfs):
    node_identifier_map = create_node_identifier_mapping(nodes_df)
    opf_keys = [k for k in sorted(opfs.keys())]
    dataframes = []
    for ok in opf_keys:
        o = opfs[ok]
        df1 = pd.DataFrame(o['res_bus'])
        df1 = df1.reset_index()
        df1['node_id'] = df1['index'].astype('int64')
        #df2 = nodes_df.reset_index()
        #df2['idx'] =df2['index'].astype('int64')
        #dataframe = df1.join(df2, on='idx')
        dataframe = df1
        dataframe['time'] = datetime.datetime.fromtimestamp(ok)
        dataframes.append(dataframe)
    res_bus_ts = pd.concat(dataframes).reset_index()[['node_id', 'p_mw', 'q_mvar', 'time']]
    res_bus_ts['identifier'] = res_bus_ts['node_id'].map(node_identifier_map)
    res_bus_ts = res_bus_ts[['p_mw', 'q_mvar', 'time', 'identifier']]
    res_bus_ts['abs_p_mw'] = abs(res_bus_ts['p_mw'])
    return res_bus_ts


In [263]:
fig = px.scatter_mapbox(nodes_df, lat="latitude", lon="longitude", color="category",  color_continuous_scale=px.colors.cyclical.IceFire, zoom=15, height=500)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [264]:
mapbox_token = ""

fig = go.Figure()

opf_keys = [k for k in sorted(opfs.keys())]
lines_df = get_line_connections(consumers, network)
load_df = create_load_dataframe(consumers, network, opfs[opf_keys[0]])
load_map_df = load_df.join(nodes_df.set_index('identifier'), on='identifier')

# add lines
fig.add_trace(go.Scattermapbox(
    mode = 'lines',
    lon = lines_df['longitude'],
    lat = lines_df['latitude']
))

# nodes
fig.add_trace(go.Scattermapbox(
    mode = "markers",
    lon = load_map_df['longitude'],
    lat = load_map_df['latitude'],
    marker = go.scattermapbox.Marker(
        color="blue"
    ),
    hoverinfo = 'text',
    text = load_map_df['name'] + '<br>' + 'load: ' + load_map_df['load'].astype(str) + " MW"
))

fig.update_layout(    
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox_style = "dark",
    mapbox = {
        'center': { 'lon': nodes_df['longitude'].mean(), 'lat': nodes_df['latitude'].mean() },
        'zoom': 15,
        'style': "open-street-map"})

fig.update_geos(projection_type='mercator')

fig.show()

['e6c8200f-d84b-499b-ad13-4313eec39ca2', '60d36bfa-92a2-4521-95fa-ed0e95f2413f', 'ec28306c-772c-4d65-9916-55c03ce61566']
['d3ae8923-d4cd-46fb-ae6c-e4a20c7e5dab']
{'from_bus': 'ec28306c-772c-4d65-9916-55c03ce61566', 'to_bus': '60d36bfa-92a2-4521-95fa-ed0e95f2413f', 'std_type': 'NAYY 4x50 SE', 'length_km': 0.1}
{'from_bus': '60d36bfa-92a2-4521-95fa-ed0e95f2413f', 'to_bus': 'e6c8200f-d84b-499b-ad13-4313eec39ca2', 'std_type': 'NAYY 4x50 SE', 'length_km': 0.1}
{'from_bus': 'e6c8200f-d84b-499b-ad13-4313eec39ca2', 'to_bus': 'd3ae8923-d4cd-46fb-ae6c-e4a20c7e5dab', 'std_type': 'NAYY 4x50 SE', 'length_km': 0.1}


In [46]:
nodes_df

,name,identifier,latitude,longitude,category
0,location 1,e6c8200f-d84b-499b-ad13-4313eec39ca2,48.3744,14.5123,household
1,location 2,60d36bfa-92a2-4521-95fa-ed0e95f2413f,48.3744,14.5115,household
2,location 3,ec28306c-772c-4d65-9916-55c03ce61566,48.3740,14.5111,household
3,grid connection,d3ae8923-d4cd-46fb-ae6c-e4a20c7e5dab,48.3744,14.5131,network


In [43]:
rline = opfs[opf_keys[0]]['res_line']
rbus = opfs[opf_keys[0]]['res_bus']
rload = opfs[opf_keys[0]]['res_load']
regrid = opfs[opf_keys[0]]['res_ext_grid']

In [44]:
rload

{'p_mw': {'0': 0.002177, '1': 0.002903, '2': 0.000235},
 'q_mvar': {'0': 0.05, '1': 0.05, '2': 0.05}}

In [45]:
rline

{'p_from_mw': {'0': -0.0002349999, '1': -0.0041496133, '2': -0.0103669092},
 'q_from_mvar': {'0': -0.0499999985, '1': -0.1001297335, '2': -0.1506510175},
 'p_to_mw': {'0': 0.001246614, '1': 0.0081899102, '2': 0.019401306},
 'q_to_mvar': {'0': 0.0501297337, '1': 0.1006510162, '2': 0.1518179322},
 'pl_mw': {'0': 0.0010116141, '1': 0.0040402969, '2': 0.0090343967},
 'ql_mvar': {'0': 0.0001297352, '1': 0.0005212827, '2': 0.0011669147},
 'i_from_ka': {'0': 0.0724742681, '1': 0.1448374191, '2': 0.2165823049},
 'i_to_ka': {'0': 0.0724727489, '1': 0.1448358949, '2': 0.2165807693},
 'i_ka': {'0': 0.0724742681, '1': 0.1448374191, '2': 0.2165823049},
 'vm_from_pu': {'0': 0.9957958942, '1': 0.998698248, '2': 1.0063622241},
 'va_from_degree': {'0': 6.7798901983, '1': 5.6246721777, '2': 3.3459509393},
 'vm_to_pu': {'0': 0.998698248, '1': 1.0063622241, '2': 1.02},
 'va_to_degree': {'0': 5.6246721777, '1': 3.3459509393, '2': 0.0},
 'loading_percent': {'0': 51.0382169481,
  '1': 101.9981824801,
  '2': 

In [ ]:
def convert_opfs_to_ts(opfs, ) -> pd.DataFrame:
    opf_keys = [k for k in sorted(opfs.keys())]
    

In [56]:
print(datetime.datetime.fromtimestamp(opf_keys[0]))
opfs[opf_keys[0]]

2023-08-21 14:00:00


{'bus': {'name': {'0': 'e6c8200f-d84b-499b-ad13-4313eec39ca2',
   '1': '60d36bfa-92a2-4521-95fa-ed0e95f2413f',
   '2': 'ec28306c-772c-4d65-9916-55c03ce61566',
   '3': 'd3ae8923-d4cd-46fb-ae6c-e4a20c7e5dab'},
  'vn_kv': {'0': 0.4, '1': 0.4, '2': 0.4, '3': 0.4},
  'type': {'0': 'b', '1': 'b', '2': 'b', '3': 'b'},
  'zone': {'0': None, '1': None, '2': None, '3': None},
  'in_service': {'0': True, '1': True, '2': True, '3': True}},
 'load': {'name': {'0': 'e6c8200f-d84b-499b-ad13-4313eec39ca2',
   '1': '60d36bfa-92a2-4521-95fa-ed0e95f2413f',
   '2': 'ec28306c-772c-4d65-9916-55c03ce61566'},
  'bus': {'0': 0, '1': 1, '2': 2},
  'p_mw': {'0': 0.002177, '1': 0.002903, '2': 0.000235},
  'q_mvar': {'0': 0.05, '1': 0.05, '2': 0.05},
  'const_z_percent': {'0': 0.0, '1': 0.0, '2': 0.0},
  'const_i_percent': {'0': 0.0, '1': 0.0, '2': 0.0},
  'sn_mva': {'0': None, '1': None, '2': None},
  'scaling': {'0': 1.0, '1': 1.0, '2': 1.0},
  'in_service': {'0': True, '1': True, '2': True},
  'type': {'0': 'wy

In [246]:
def line_ts(nodes_df, opfs):
    node_identifier_map = create_node_identifier_mapping(nodes_df)
    opf_keys = [k for k in sorted(opfs.keys())]
    dataframes = []
    for ok in opf_keys:
        o = opfs[ok]
        df1 = pd.DataFrame(o['res_line'])
        df1 = df1.reset_index()
        df1['node_id'] = df1['index'].astype('int64')
        dataframe = df1
        dataframe['time'] = datetime.datetime.fromtimestamp(ok)
        dataframes.append(dataframe)
    res_line_ts = pd.concat(dataframes).reset_index()#[['node_id', 'p_mw', 'q_mvar', 'time']]
    #res_line_ts['identifier'] = res_line_ts['node_id'].map(node_identifier_map)
    #res_line_ts = res_line_ts[['p_mw', 'q_mvar', 'time', 'identifier']]
    return res_line_ts

In [248]:
opfs[ok]['res_line']


{'p_from_mw': {'0': -0.0020739999, '1': -0.0054593879, '2': -0.0109209071},
 'q_from_mvar': {'0': -0.0499999984, '1': -0.100130353, '2': -0.1506530892},
 'p_to_mw': {'0': 0.0030903887, '1': 0.0095109081, '2': 0.0199642279},
 'q_to_mvar': {'0': 0.0501303533, '1': 0.1006530878, '2': 0.1518211578},
 'pl_mw': {'0': 0.0010163888, '1': 0.0040515201, '2': 0.0090433208},
 'ql_mvar': {'0': 0.0001303549, '1': 0.0005227347, '2': 0.0011680687},
 'i_from_ka': {'0': 0.0726450989, '1': 0.1450384439, '2': 0.2166892462},
 'i_to_ka': {'0': 0.0726435833, '1': 0.1450369217, '2': 0.2166877114},
 'i_ka': {'0': 0.0726450989, '1': 0.1450384439, '2': 0.2166892462},
 'vm_from_pu': {'0': 0.9942975115, '1': 0.9979448955, '2': 1.0061393975},
 'va_from_degree': {'0': 6.7745367462, '1': 5.6222137558, '2': 3.3451320878},
 'vm_to_pu': {'0': 0.9979448955, '1': 1.0061393975, '2': 1.02},
 'va_to_degree': {'0': 5.6222137558, '1': 3.3451320878, '2': 0.0},
 'loading_percent': {'0': 51.1585203623,
  '1': 102.1397492189,
  '2

In [249]:
opfs[ok]['res_load']

{'p_mw': {'0': 0.00141, '1': 0.002369, '2': 0.002074},
 'q_mvar': {'0': 0.05, '1': 0.05, '2': 0.05}}

In [260]:
print(json.dumps(opfs[ok]['res_line'], indent=2))

{
  "p_from_mw": {
    "0": -0.0020739999,
    "1": -0.0054593879,
    "2": -0.0109209071
  },
  "q_from_mvar": {
    "0": -0.0499999984,
    "1": -0.100130353,
    "2": -0.1506530892
  },
  "p_to_mw": {
    "0": 0.0030903887,
    "1": 0.0095109081,
    "2": 0.0199642279
  },
  "q_to_mvar": {
    "0": 0.0501303533,
    "1": 0.1006530878,
    "2": 0.1518211578
  },
  "pl_mw": {
    "0": 0.0010163888,
    "1": 0.0040515201,
    "2": 0.0090433208
  },
  "ql_mvar": {
    "0": 0.0001303549,
    "1": 0.0005227347,
    "2": 0.0011680687
  },
  "i_from_ka": {
    "0": 0.0726450989,
    "1": 0.1450384439,
    "2": 0.2166892462
  },
  "i_to_ka": {
    "0": 0.0726435833,
    "1": 0.1450369217,
    "2": 0.2166877114
  },
  "i_ka": {
    "0": 0.0726450989,
    "1": 0.1450384439,
    "2": 0.2166892462
  },
  "vm_from_pu": {
    "0": 0.9942975115,
    "1": 0.9979448955,
    "2": 1.0061393975
  },
  "va_from_degree": {
    "0": 6.7745367462,
    "1": 5.6222137558,
    "2": 3.3451320878
  },
  "vm_to_p

In [262]:
print(json.dumps(opfs[ok]['line'], indent=2))

{
  "name": {
    "0": null,
    "1": null,
    "2": null
  },
  "std_type": {
    "0": "NAYY 4x50 SE",
    "1": "NAYY 4x50 SE",
    "2": "NAYY 4x50 SE"
  },
  "from_bus": {
    "0": 2,
    "1": 1,
    "2": 0
  },
  "to_bus": {
    "0": 1,
    "1": 0,
    "2": 3
  },
  "length_km": {
    "0": 0.1,
    "1": 0.1,
    "2": 0.1
  },
  "r_ohm_per_km": {
    "0": 0.642,
    "1": 0.642,
    "2": 0.642
  },
  "x_ohm_per_km": {
    "0": 0.083,
    "1": 0.083,
    "2": 0.083
  },
  "c_nf_per_km": {
    "0": 210.0,
    "1": 210.0,
    "2": 210.0
  },
  "g_us_per_km": {
    "0": 0.0,
    "1": 0.0,
    "2": 0.0
  },
  "max_i_ka": {
    "0": 0.142,
    "1": 0.142,
    "2": 0.142
  },
  "df": {
    "0": 1.0,
    "1": 1.0,
    "2": 1.0
  },
  "parallel": {
    "0": 1,
    "1": 1,
    "2": 1
  },
  "type": {
    "0": "cs",
    "1": "cs",
    "2": "cs"
  },
  "in_service": {
    "0": true,
    "1": true,
    "2": true
  }
}


In [253]:
res_line_ts

,level_0,index,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent,node_id,time
0,0,0,-0.000235,-0.050000,0.001247,0.050130,0.001012,0.000130,0.072474,0.072473,0.072474,0.995796,6.779890,0.998698,5.624672,51.038217,0,2023-08-21 14:00:00
1,1,1,-0.004150,-0.100130,0.008190,0.100651,0.004040,0.000521,0.144837,0.144836,0.144837,0.998698,5.624672,1.006362,3.345951,101.998182,1,2023-08-21 14:00:00
2,2,2,-0.010367,-0.150651,0.019401,0.151818,0.009034,0.001167,0.216582,0.216581,0.216582,1.006362,3.345951,1.020000,0.000000,152.522750,2,2023-08-21 14:00:00
3,0,0,-0.000830,-0.050000,0.001843,0.050130,0.001013,0.000130,0.072528,0.072527,0.072528,0.995180,6.777952,0.998324,5.623366,51.076280,0,2023-08-21 14:15:00
4,1,1,-0.004835,-0.100130,0.008881,0.100652,0.004046,0.000522,0.144936,0.144935,0.144936,0.998324,5.623366,1.006265,3.345593,102.067949,1,2023-08-21 14:15:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,1,1,-0.005736,-0.100130,0.009787,0.100653,0.004051,0.000523,0.145036,0.145034,0.145036,0.998118,5.621455,1.006422,3.346229,102.137683,1,2023-08-22 13:30:00
284,2,2,-0.010218,-0.150653,0.019251,0.151820,0.009032,0.001167,0.216558,0.216556,0.216558,1.006422,3.346229,1.020000,0.000000,152.505533,2,2023-08-22 13:30:00
285,0,0,-0.002074,-0.050000,0.003090,0.050130,0.001016,0.000130,0.072645,0.072644,0.072645,0.994298,6.774537,0.997945,5.622214,51.158520,0,2023-08-22 13:45:00
286,1,1,-0.005459,-0.100130,0.009511,0.100653,0.004052,0.000523,0.145038,0.145037,0.145038,0.997945,5.622214,1.006139,3.345132,102.139749,1,2023-08-22 13:45:00


In [254]:
#res_bus_ts = bus_ts(nodes_df, opfs)
#res_load_ts = load_ts(nodes_df, opfs)
res_line_ts = line_ts(nodes_df, opfs)
px.line(res_line_ts, x='time', y='p_from_mw', color='index')


In [291]:
nodes_df

,name,identifier,latitude,longitude,category
0,location 1,e6c8200f-d84b-499b-ad13-4313eec39ca2,48.3744,14.5123,household
1,location 2,60d36bfa-92a2-4521-95fa-ed0e95f2413f,48.3744,14.5115,household
2,location 3,ec28306c-772c-4d65-9916-55c03ce61566,48.3740,14.5111,household
3,grid connection,d3ae8923-d4cd-46fb-ae6c-e4a20c7e5dab,48.3744,14.5131,network


In [290]:
inm = create_identifier_name_mapping(nodes_df)
load_nodes = set(res_load_ts['identifier'])
print(load_nodes)
fig_load = go.Figure()
for ln in load_nodes:
    data = res_load_ts[res_load_ts['identifier']==ln]
    print(data.head())
    fig_load.add_trace(go.Scatter(x=data['time'], y=data['p_mw'],
                    mode='markers+lines',
                    name=nimap[ln]))
#res_load_ts
fig_load.show()

{0: 'e6c8200f-d84b-499b-ad13-4313eec39ca2', 1: '60d36bfa-92a2-4521-95fa-ed0e95f2413f', 2: 'ec28306c-772c-4d65-9916-55c03ce61566', 3: 'd3ae8923-d4cd-46fb-ae6c-e4a20c7e5dab'}
{'60d36bfa-92a2-4521-95fa-ed0e95f2413f', 'ec28306c-772c-4d65-9916-55c03ce61566', 'e6c8200f-d84b-499b-ad13-4313eec39ca2'}
        p_mw  q_mvar                time                            identifier
1   0.002903    0.05 2023-08-21 14:00:00  60d36bfa-92a2-4521-95fa-ed0e95f2413f
4   0.002992    0.05 2023-08-21 14:15:00  60d36bfa-92a2-4521-95fa-ed0e95f2413f
7   0.000109    0.05 2023-08-21 14:30:00  60d36bfa-92a2-4521-95fa-ed0e95f2413f
10  0.000118    0.05 2023-08-21 14:45:00  60d36bfa-92a2-4521-95fa-ed0e95f2413f
13  0.002063    0.05 2023-08-21 15:00:00  60d36bfa-92a2-4521-95fa-ed0e95f2413f


KeyError: '60d36bfa-92a2-4521-95fa-ed0e95f2413f'

In [147]:
tdf = df1.join(df2, on='index')#, lsuffix='_l', rsuffix='_r')
tdf['time'] =

SyntaxError: invalid syntax (3393917304.py, line 2)

In [124]:
df1

,index,p_mw,q_mvar
0,0,0.002177,0.05
1,1,0.002903,0.05
2,2,0.000235,0.05


In [125]:
df2

,index,name,identifier,latitude,longitude,category
0,0,location 1,e6c8200f-d84b-499b-ad13-4313eec39ca2,48.3744,14.5123,household
1,1,location 2,60d36bfa-92a2-4521-95fa-ed0e95f2413f,48.3744,14.5115,household
2,2,location 3,ec28306c-772c-4d65-9916-55c03ce61566,48.3740,14.5111,household
3,3,grid connection,d3ae8923-d4cd-46fb-ae6c-e4a20c7e5dab,48.3744,14.5131,network
